In [ ]:
#!pip install selenium
#!pip install requests_html

from requests_html import HTMLSession,AsyncHTMLSession  
from bs4 import BeautifulSoup as bs 
import pandas as pd
import json
import re
import time
from tqdm import tqdm
import numpy as np
import requests
from datetime import date,datetime
from sqlalchemy import create_engine
from sqlalchemy.ext.compiler import compiles
from sqlalchemy.sql.expression import Insert
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Chrome
import datetime as dt

url  = input('채널 URL을 입력해주세요 : ',) # 첫 화면 URL

driver = Chrome()
driver.implicitly_wait(3)
driver.get(url)  
driver.maximize_window()

driver.find_element_by_xpath('//*[@id="tabsContent"]/tp-yt-paper-tab[2]/div').click() # 채널 동영상 탭

body = driver.find_element_by_tag_name('body')

# 스크롤 다운
num_of_pagedowns = 10

while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(3)
    num_of_pagedowns -= 1
    
html0 = driver.page_source # 현재 페이지 html
html = bs(html0,'html.parser')


video_url_list = []

for i in tqdm(range(0,120)): # 동영상 수 범위 설정
    video_url = html.find_all('a', {'id' : 'video-title'})[i]['href']
    video_url_list.append(video_url)
    
data = pd.DataFrame()
data['video_url'] = video_url_list
data.drop_duplicates(['video_url'], inplace = True) # 중복 수집되는 아이디 제거